## 📌 **Manejo Avanzado de Bases de Datos en Django**  

El objetivo de este tema es que comprendas cómo trabajar con bases de datos en Django usando ORM (**Object-Relational Mapping**). Vamos a profundizar en **filtros, agregaciones y relaciones con `ForeignKey`** para que puedas hacer consultas avanzadas de manera eficiente.  

---

## 🔹 **1. ¿Cómo interactúa Django con la base de datos?**  

Django utiliza un sistema llamado **ORM (Object-Relational Mapping)**, que permite interactuar con la base de datos usando Python en lugar de SQL puro.  

📌 **Ventajas del ORM de Django:**  
✔️ Evita escribir SQL manualmente.  
✔️ Permite cambiar de base de datos fácilmente (SQLite, PostgreSQL, MySQL).  
✔️ Facilita la validación y manipulación de datos.  

📍 **Ejemplo básico:**  

En vez de escribir SQL:  

```sql
SELECT * FROM usuarios WHERE edad > 18;
```

En Django usamos:  

```python
Usuarios.objects.filter(edad__gt=18)
```

Esto hace el código más legible y seguro.  

---

## 🔹 **2. Creando modelos con `ForeignKey` y relaciones**  

Imaginemos que tenemos un sistema de blog con usuarios y publicaciones. Creamos dos modelos en `models.py`:  

```python
from django.db import models

class Usuario(models.Model):
    nombre = models.CharField(max_length=100)
    email = models.EmailField(unique=True)

    def __str__(self):
        return self.nombre

class Publicacion(models.Model):
    titulo = models.CharField(max_length=200)
    contenido = models.TextField()
    fecha_creacion = models.DateTimeField(auto_now_add=True)
    autor = models.ForeignKey(Usuario, on_delete=models.CASCADE)

    def __str__(self):
        return self.titulo
```

📌 **Explicación:**  
✔️ `Usuario` representa a un usuario con `nombre` y `email`.  
✔️ `Publicacion` tiene un `titulo`, `contenido`, y está relacionada con un usuario usando `ForeignKey`.  
✔️ `on_delete=models.CASCADE` significa que si un usuario se elimina, sus publicaciones también se eliminan.  

📍 **Generamos las migraciones y aplicamos los cambios:**  

```bash
python manage.py makemigrations
python manage.py migrate
```

---

## 🔹 **3. Consultas avanzadas con filtros (`filter()`, `exclude()`, `get()`)**  

### 📍 **Filtrar registros**  

Obtener todas las publicaciones de un usuario:  

```python
usuario = Usuario.objects.get(nombre="Carlos")
publicaciones = Publicacion.objects.filter(autor=usuario)
```

Filtrar publicaciones creadas después de cierta fecha:  

```python
from django.utils import timezone
publicaciones = Publicacion.objects.filter(fecha_creacion__gte=timezone.now() - timezone.timedelta(days=7))
```

📌 **Operadores de consulta:**  
✔️ `__gte` → Mayor o igual (`fecha_creacion__gte=...`)  
✔️ `__lte` → Menor o igual (`fecha_creacion__lte=...`)  
✔️ `__contains` → Contiene una palabra (`titulo__contains="Django"`)  
✔️ `__exact` → Coincidencia exacta (`email__exact="test@email.com"`)  

---

### 📍 **Excluir registros**  

Obtener todas las publicaciones **excepto** las del usuario "Carlos":  

```python
publicaciones = Publicacion.objects.exclude(autor__nombre="Carlos")
```

---

### 📍 **Obtener un único registro con `get()`**  

Si sabemos que solo hay un resultado, usamos `get()`:  

```python
usuario = Usuario.objects.get(email="carlos@email.com")
```

⚠️ **Precaución**: Si `get()` no encuentra un resultado, lanza un error (`DoesNotExist`).  

---

## 🔹 **4. Relaciones inversas (`related_name`)**  

Django permite acceder a las relaciones inversas automáticamente.  

📍 **Ejemplo:** Obtener todas las publicaciones de un usuario **desde el modelo `Usuario`**:  

```python
usuario = Usuario.objects.get(nombre="Carlos")
publicaciones = usuario.publicacion_set.all()  # Django agrega "_set" a los modelos relacionados
```

Podemos cambiar `_set` con `related_name` en `models.py`:  

```python
class Publicacion(models.Model):
    autor = models.ForeignKey(Usuario, on_delete=models.CASCADE, related_name="publicaciones")
```

Ahora accedemos así:  

```python
publicaciones = usuario.publicaciones.all()
```

✔️ **Más legible y semántico**.  

---

## 🔹 **5. Agregaciones (`aggregate()` y `annotate()`)**  

Django permite hacer cálculos como contar registros o sumar valores.  

### 📍 **Ejemplo: Contar publicaciones por usuario**  

```python
from django.db.models import Count

usuarios = Usuario.objects.annotate(num_publicaciones=Count('publicaciones'))

for usuario in usuarios:
    print(f"{usuario.nombre} tiene {usuario.num_publicaciones} publicaciones.")
```

📌 **Otros ejemplos de agregaciones:**  
✔️ `Sum('campo')` → Sumar valores de un campo.  
✔️ `Avg('campo')` → Promediar valores.  
✔️ `Max('campo')` → Obtener el máximo.  
✔️ `Min('campo')` → Obtener el mínimo.  

---

## 🔹 **6. Uso de `select_related()` y `prefetch_related()` para optimizar consultas**  

Cuando trabajamos con relaciones `ForeignKey`, Django hace consultas adicionales a la base de datos. Podemos optimizar esto con:  

### 📍 **`select_related()`**  
Usado en relaciones **uno a uno** (`OneToOneField`) y **muchos a uno** (`ForeignKey`), evitando consultas extra.  

```python
publicaciones = Publicacion.objects.select_related('autor').all()
```

📌 **Beneficio:** Recupera los datos en una sola consulta SQL en lugar de múltiples.  

---

### 📍 **`prefetch_related()`**  
Usado en relaciones **muchos a muchos** (`ManyToManyField`) y **uno a muchos** (`ForeignKey` en sentido inverso).  

```python
usuarios = Usuario.objects.prefetch_related('publicaciones').all()
```

📌 **Beneficio:** Hace múltiples consultas, pero las optimiza internamente.  

---

## ✅ **Resumen del flujo de trabajo con bases de datos en Django**  

1️⃣ Definimos modelos en `models.py` y aplicamos migraciones.  
2️⃣ Usamos `filter()`, `exclude()`, `get()` para consultar datos.  
3️⃣ Aprovechamos relaciones inversas con `related_name`.  
4️⃣ Hacemos cálculos con `aggregate()` y `annotate()`.  
5️⃣ Optimizamos consultas con `select_related()` y `prefetch_related()`.  